<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [2]:
import numpy as np

In [7]:
# Define input and output
X = np.array(([0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1],
              [0, 0, 0]), dtype = float)

y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype = float)

In [8]:
class MultilayerPerceptron:
    def __init__(self):
        # Set up architecture
        self.inputs = 3
        self.hiddenNodes = 4
        self.output = 1
        
        # Initialize Weights
        # 3x4 for: Inputs --> Hidden Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        
        # 4x1 for: Hidden Layer --> Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.output)
        
    # Define activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # Slope of activation function
    def sigmoidPrime(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)
    
    # Feed forward / prediction method
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka 'predict'
        """
        
        # Weighted sum of inputs
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of Weighted Sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Next layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final node - activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
    
    def backward(self, X, y, y_pred):
        """
        Back propagate through the network
        """
        
        # Error in output
        self.output_error = y - y_pred
        
        
        # hidden --> Output
        # Apply Derivative of Sigmoid to error
        self.output_delta = self.output_error * self.sigmoidPrime(y_pred)
        
        # z2 Error
        self.z2_error = self.output_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Update weights
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjust second set of weights
        self.weights2 += self.activated_hidden.T.dot(self.output_delta)
        
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [11]:
# Train my 'net
nn = MultilayerPerceptron()

# Set number of Epochs
for i in range(100000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 10000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.84060962]
 [0.8728097 ]
 [0.8564305 ]
 [0.83216848]
 [0.80723623]
 [0.88213311]
 [0.7825763 ]]
Loss: 
 0.3141891293154428
+---------EPOCH 2---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.72794879]
 [0.75666393]
 [0.7408808 ]
 [0.72102932]
 [0.70062363]
 [0.76518521]
 [0.68326912]]
Loss: 
 0.26801151313942123
+---------EPOCH 3---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.64320363]
 [0.66252191]
 [0.65141543]
 [0.63785   ]
 [0.62435046]
 [0.66813223]
 [0.61373475]]
Loss: 
 0.24920711410330018
+---------EPOC

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [13]:
!pip install tensorflow

In [4]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

## Using Tensorflow

In [1]:
import tensorflow as tf

tf.__version__

'1.14.0'

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
import matplotlib.pyplot as plt

In [7]:
# plt.imshow(x_train[0])
# print(x_train[0])

In [8]:
# Normalize Data
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [9]:
# Define Architecture #
model = tf.keras.models.Sequential()

# Flatten Data - Input Layer
model.add(tf.keras.layers.Flatten())

# Two hidden layers
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# Training Params
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train Model
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 6s 97us/sample - loss: 0.2647 - acc: 0.9229
Epoch 2/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.1063 - acc: 0.9676
Epoch 3/5
46720/60000 [======================>.......] - ETA: 1s - loss: 0.0698 - acc: 0.9782

In [ ]:
# Validation loss and accuracy
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)

In [ ]:
predictions = model.predict([x_test])

In [ ]:
print(predictions)

In [ ]:
np.argmax(predictions[100])

In [ ]:
plt.imshow(x_test[100])

In [ ]:
# # Pickle
# model.save('num_reader.model')
# new_model = tf.keras.models.load_model('num_reader.model')

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?